<a href="https://colab.research.google.com/github/tanuja1708/EEG-emotions/blob/main/Final_Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install flask
!npm install -g localtunnel
!pip install flask flask-ngrok tensorflow joblib pykalman scipy
!pip install PyWavelets

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙
added 22 packages in 3s
⠙
⠙3 packages are looking for funding
⠙  run `npm fund` for details
⠙Requirement already satisfied: flask in /usr/local/lib/python3.11/dist-packages (3.1.1)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 248.5/248.5 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.7/142.7 kB 15.0 MB/s eta 0:00:00


In [2]:
#with bandpass with ui

from flask import Flask, jsonify
import numpy as np
from tensorflow.keras.models import load_model
import joblib
from scipy.signal import butter, filtfilt,welch
from pykalman import KalmanFilter
import subprocess
import pywt
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

app = Flask(__name__)

sampling_rate_original = 2500
sampling_rate_final = 200
segment_length = sampling_rate_final * 1

freq_bands = {
    "Delta": (1, 4),
    "Theta": (4, 8),
    "Alpha": (8, 14),
    "Beta": (14, 31),
    "Gamma": (31, 50)
}

# --- Kalman Filter Setup ---
kf = KalmanFilter(
    transition_matrices=[1],
    observation_matrices=[1],
    transition_covariance=0.2 * np.eye(1),
    observation_covariance=0.5 * np.eye(1),
    initial_state_mean=[20],
    initial_state_covariance=5 * np.eye(1)
)

# --- Filtering & Downsampling ---
def bandpass_filter(data, lowcut, highcut, fs):
    nyq = 0.5 * fs
    b, a = butter(4, [lowcut / nyq, highcut / nyq], btype='band')
    return filtfilt(b, a, data)

def downsample(signal, original_fs, target_fs):
    factor = int(original_fs / target_fs)
    return signal[::factor]

# --- EEG Generator ---
def generate_eeg():
    t = np.linspace(0, 1, sampling_rate_original)
    eeg = []
    for _ in range(62):
        alpha = np.random.uniform(40, 80) * np.sin(2 * np.pi * np.random.uniform(8, 13) * t)
        beta = np.random.uniform(15, 35) * np.sin(2 * np.pi * np.random.uniform(14, 30) * t)
        delta = np.random.uniform(1, 5) * np.sin(2 * np.pi * np.random.uniform(1, 3) * t)
        theta = np.random.uniform(1, 5) * np.sin(2 * np.pi * np.random.uniform(4, 7) * t)
        gamma = np.random.uniform(1, 5) * np.sin(2 * np.pi * np.random.uniform(31, 50) * t)
        noise = np.random.normal(0, 3, t.shape)
        signal = delta + theta + alpha + beta + gamma + noise
        eeg.append(np.clip(signal, -70, 70))
    return np.array(eeg)

# --- MSPCA Denoising ---
def mspca_denoise(eeg_data, wavelet='db4', level=4):
    denoised_eeg = []
    for channel in eeg_data:
        coeffs = pywt.wavedec(channel, wavelet, level=level)
        denoised_coeffs = []
        for c in coeffs:
            if len(c) > 1:
                c = c.reshape(-1, 1)
                pca = PCA(n_components=1)
                transformed = pca.fit_transform(c)
                reconstructed = pca.inverse_transform(transformed).flatten()
                denoised_coeffs.append(reconstructed)
            else:
                denoised_coeffs.append(c)
        denoised_signal = pywt.waverec(denoised_coeffs, wavelet)
        denoised_eeg.append(denoised_signal[:len(channel)])
    return np.array(denoised_eeg)

# --- DE_LDS with Band-Split, Welch & MSPCA ---
def extract_de_lds(eeg_data):
    # 1) Downsample all channels
    eeg_ds = [downsample(ch, sampling_rate_original, sampling_rate_final)
              for ch in eeg_data]

    de_features = []  # will become list of 62 vectors of length 5
    for ch in eeg_ds:
        channel_de = []

        for low, high in freq_bands.values():
            # 2) Narrow-band filter
            sig_band = bandpass_filter(ch, low, high, sampling_rate_final)


            # 3) MSPCA denoise _this_ 1D signal
            denoised = mspca_denoise([sig_band])[0]

            # 4) PSD → DE
            freqs, psd = welch(denoised, fs=sampling_rate_final,
                               nperseg=min(100, segment_length))
            P_mean = np.mean(psd)
            de_val = 0.5 * np.log(2 * np.pi * np.e * (P_mean + 1e-6))
            channel_de.append(de_val)

        # 5) Kalman smooth the 5-band DE vector
        smoothed, _ = kf.filter(np.array(channel_de).reshape(-1, 1))
        de_features.append(smoothed.flatten())

    # 6) Normalize & flatten
    feats = np.array(de_features)            # shape (62,5)
    mn, mx = feats.min(), feats.max()
    scaled = 15 + (feats - mn) / (mx - mn + 1e-8) * (27 - 15)
    # print(scaled.flatten())
    return scaled.flatten()

@app.route('/predict', methods=['GET'])
def predict_emotion():
    eeg_data = generate_eeg()
    features = extract_de_lds(eeg_data).reshape((1, 1,310))

    # Load model and encoder
    model = load_model('/content/eeg_emotion_gan_model.h5')
    encoder = joblib.load('/content/eeg_label_encoder.pkl')

    prediction = model.predict(features)
    label = encoder.inverse_transform(np.argmax(prediction, axis=1))[0]

    print(f"Predicted Emotion: {label}")
    return jsonify({'emotion': label, 'eeg_data': eeg_data.tolist()})

# --- LocalTunnel Setup ---
def run_localtunnel():
    subdomain = "eegemotion"
    process = subprocess.Popen(['lt', '--port', '5000', '-s', subdomain], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    for line in process.stdout:
        print(line.decode('utf-8').strip())

# --- Run App ---
if __name__ == "__main__":
    from threading import Thread
    thread = Thread(target=run_localtunnel)
    thread.start()
    app.run(port=5000)




 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


your url is: https://eegemotion.loca.lt


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 840ms/step


INFO:werkzeug:127.0.0.1 - - [18/May/2025 17:39:35] "GET /predict HTTP/1.1" 200 -


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 516ms/step
Predicted Emotion: Happy


INFO:werkzeug:127.0.0.1 - - [18/May/2025 17:41:32] "GET /predict HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 301ms/step


INFO:werkzeug:127.0.0.1 - - [18/May/2025 17:41:43] "GET /predict HTTP/1.1" 200 -


Predicted Emotion: Anger
